In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
with open("MW.txt") as data:
    reader = csv.reader(data, delimiter='\t')
    d = list(reader)

for i in d:
    i[6]=[int(i) for i in  i[6].split(',')]
    i[6]=i[6][0:441]

In [3]:
j=1000
for m in d:
    if int(m[5])<j:
        j=int(m[5])
print(j)
print(d[1])

441
['1', '1', 'MW', 'FP1', '1', '889', [83, 74, 65, 65, 66, 55, 43, 25, 18, 20, 26, 33, 41, 38, 28, 36, 55, 60, 56, 58, 72, 85, 76, 69, 74, 83, 86, 88, 97, 112, 114, 109, 99, 76, 54, 50, 66, 86, 97, 99, 96, 86, 82, 73, 69, 70, 70, 60, 65, 71, 69, 64, 57, 54, 56, 58, 50, 20, -20, -56, -74, -69, -57, -57, -60, -69, -75, -71, -59, -45, -24, 3, 13, 10, 8, 10, 5, 7, 8, -3, -22, -28, -20, 1, 3, -10, -18, -12, -19, -23, -25, -36, -45, -43, -37, -23, -4, 4, -2, -3, -3, -2, 5, 8, 7, 5, 16, 29, 34, 35, 36, 48, 68, 80, 80, 75, 68, 69, 77, 89, 105, 100, 80, 64, 56, 59, 64, 57, 49, 42, 48, 60, 75, 77, 70, 61, 45, 28, 24, 28, 29, 34, 40, 50, 42, 27, 16, 4, 1, 6, 12, 16, 20, 24, 34, 44, 54, 58, 53, 40, 24, 11, 4, -4, -14, -34, -56, -76, -77, -68, -53, -44, -38, -35, -30, -30, -30, -25, -6, 18, 29, 27, 23, 27, 41, 50, 51, 57, 57, 51, 55, 57, 58, 53, 37, 20, 9, 12, 29, 49, 49, 37, 28, 37, 57, 76, 83, 74, 68, 50, 25, 9, 13, 39, 71, 103, 121, 130, 132, 136, 146, 163, 184, 188, 171, 152, 152, 164, 166, 1

In [4]:
dataset = pd.DataFrame(d)
X = dataset.iloc[:, 6].values
y = dataset.iloc[:, 4].values
x=[]

for i in X:
    x.append(i)
X = pd.DataFrame(x)
print(y.shape)
X.shape

(67635,)


(67635, 441)

In [5]:
print(X.T[0])
print(y[0])


0      38
1      48
2      51
3      44
4      48
       ..
436   -62
437   -60
438   -53
439   -55
440   -73
Name: 0, Length: 441, dtype: int64
0


In [6]:
print(X.T[0])
X.isnull().sum()

0      38
1      48
2      51
3      44
4      48
       ..
436   -62
437   -60
438   -53
439   -55
440   -73
Name: 0, Length: 441, dtype: int64


0      0
1      0
2      0
3      0
4      0
      ..
436    0
437    0
438    0
439    0
440    0
Length: 441, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import *
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(np.shape(X_train))

X_train=np.array(X_train)
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=7, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=10, activation='relu'))
model.add(AveragePooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=7, activation='relu'))
model.add(LSTM(441, activation='relu', return_sequences=True,input_shape=(441, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy'])
model.summary()


(47344, 441)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 441, 441)          781452    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 432, 64)           282304    
_________________________________________________________________
average_pooling1d_2 (Average (None, 216, 64)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 64)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 210, 128)          57472     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 204, 128)          114816    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 102, 

In [11]:


X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
model.fit(X_train, y_train, epochs = 10, batch_size = 10)

X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))
predicted_stock_price = model.evaluate(X_test,y_test)

print(predicted_stock_price)
        

Epoch 1/10
  630/47344 [..............................] - ETA: 24:42 - loss: -37.0188 - accuracy: 0.090

KeyboardInterrupt: 